# 이진화 전처리 함수

In [3]:
from cv2 import *
from keras.preprocessing.image import img_to_array, load_img

Using TensorFlow backend.


In [75]:
# 1. 원본 그대로~
def original(img_path):
    img = load_img(img_path)
    img_array = img_to_array(img)
    bgr_image_array = cvtColor(img_array, COLOR_RGB2BGR)
    return bgr_image_array

In [76]:
def gray_scaling(img_path):
  img = load_img(img_path)
  img_array = img_to_array(img)
  gray_img_array = cvtColor(img_array, COLOR_BGR2GRAY)
  gray_img_array = gray_img_array.astype("uint8")
  return gray_img_array

In [77]:
# 2. adaptive threshold
def thresholding(img_path):
    # grayscale 변환
    gray_img_array = gray_scaling(img_path)
    
    # adaptive threshold 적용
    thresholded_img_array = cv2.adaptiveThreshold(gray_image_array,
                               255,
                               ADAPTIVE_THRESH_GAUSSIAN_C, 
                               THRESH_BINARY,
                               15,
                               7)
    
    # bgr 변환
    bgr_image_array = cvtColor(thresholded_img_array,COLOR_GRAY2BGR)
    return bgr_image_array

In [70]:
# 3. edge_detection
def edge_detecting(img_path):
    # grayscale 변환
    gray_img_array = gray_scaling(img_path)

    # edge_detecting
    img_sobel_x = Sobel(gray_img_array, CV_64F, 1, 0, ksize=3)
    img_sobel_x = convertScaleAbs(img_sobel_x)

    img_sobel_y = Sobel(gray_img_array, CV_64F, 0, 1, ksize=3)
    img_sobel_y = convertScaleAbs(img_sobel_y)

    img_sobel = addWeighted(img_sobel_x,1,img_sobel_y,1,0)

    # bgr로 변환
    bgr_image_array = cvtColor(img_sobel, COLOR_GRAY2BGR)
    return bgr_image_array

# 원본 이미지 경로 불러오기

In [71]:
import glob

In [79]:
img_pathes = glob.glob("/content/drive/Shared drives/선빵팀 :) 2/TuningStar_cate/*.jpg")
img_pathes = sorted(img_pathes) # alphanumeric order로 정렬.

# 분류할 카테고리 넘버 리스트, 딕셔너리 만들기
#### category_nums, category_count_dict

In [95]:
# img_pathes에 있는 각 이미지 별로 어떤 카테고리에 속하는 지를 따로 리스트로 저장.
img_category_list=[]
for img_path in img_pathes:
    img_category = img_path.split("/")[-1].split(" ")[0] # 휠 파일 이름.jpg
    img_category_list.append(int(img_category))

전체 이미지 중에서 각 카테고리 번호가 몇 개씩 들어있는 지 카운팅한 뒤, 카테고리(key):등장횟수(value) 형태로 딕셔너리 생성

In [96]:
category_counting = {}
for category_number in list(set(img_category_list)):
    num_count = img_category_list.count(category_number)
    category_counting[category_number] = num_count

Keras의 ImageDataGenerator를 통해 Data Augmentation을 할 수 있는 generator생성.

# 각 전처리 방법으로 이진화 후 다른 폴더에 저장

배정받은 이미지 경로들을 통해 이미지를 가져오고,
하나의 카테고리를 하나의 폴더로 만듦. 위에서도 이야기했듯이 파일 번호 = 카테고리이므로 이미 카테고리 폴더가 생성되어있는 경우는 넘어가게 햇음.

In [100]:
def category_calculating(img_path):
    global category_count_dict
    # input image가 속한 카테고리 구하기
    img_category = img_path.split("/")[-1].split(" ")[0] # 휠 파일 이름.jpg
    img_category = int(img_category)

    # input image가 속한 카테고리가 총 몇 개 있는 지 확인(카테고리 별로 같은 크기로 Augmentation하기 위해서는, 각 카테고리에 속한 image 수로 전체 목표 Augmentation수를 나누어야 하기 때문)
    category_count = category_count_dict[img_category]
    return img_category, category_count

# 이미지 전처리 함수
#: 앞서 정해준 이진화 방법들 중 하나를 function에 넣어주면 이미지를 해당 방법으로 전처리. 
def preprocessing(img_path, function):
    preprocessed_img = function(img_path) # function에 전처리 함수를 넣어 해당 방법으로 전처리한다.
    input_img = expand_dims(img,0) # 차원 추가
    return input_img

In [102]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
''' 전처리 함수 메뉴
1. original : 원본 이미지
2. thresholding : Adaptive threshold(GAUSSIAN_WINDOW)
3. edge_detecting : Sobel
'''

with tf.device('/device:GPU:0'):
    ##ImageDataGenerator에 원하는 옵션들을 추가해주면 옵션들 적용하면서 랜덤하게 생성해줍니다.
    datagen = ImageDataGenerator(
            rotation_range = 90,
            width_shift_range = 0.2,
            height_shift_range = 0.2,
            brightness_range = [0.05 , 2.0],
            vertical_flip = True,
            horizontal_flip = True)
    
    function = original  # original, thresholding, edge_detecting 중 택 1.

    for img_path in img_pathes:

        ''' category별 개수 구하기 '''
        img_category, category_count = category_calculating(img_path)

        ''' 전처리 '''
        generator_input_img = preprocessing(img_path, function)

        ''' Augment & Save '''
        # augmentation 개수
        img_num = 1800 # 1680 : 210의 배수로 해줘야 각 폴더에 들어가는 이미지 수가 최대한 비슷해짐
        aug_num = int(img_num/category_count)
 
        folder_path = "/content/Training/" + str(img_category) # folder_path는 자기가 저장할 경로에 맞게 수정해주어야 함. str(category)는 건들지 말고 앞부분만 수정.
        if not(os.path.isdir(folder_path)):
            os.makedirs(folder_path)
        j=0
        for batch in datagen.flow(x,
                                  batch_size=1,
                                  save_to_dir=folder_path,
                                  save_prefix=img_category,
                                  save_format="jpeg"):
            j+=1
            if j > aug_num:
                break

0 _category count :  4
0 _aug_num :  450
0 _category count :  4
0 _aug_num :  450
0 _category count :  4
0 _aug_num :  450
0 _category count :  4
0 _aug_num :  450
1 _category count :  3
1 _aug_num :  600
1 _category count :  3
1 _aug_num :  600
1 _category count :  3
1 _aug_num :  600
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
10 _category count :  12
10 _aug_num :  150
11 _category count :  4
11 _aug_num :  450
11 _category count :  4
11 _aug_num :  450
11 _category count :  4
11 _aug_num :  450
11 _category count :  4
11 _aug_num :  450
12 _category 

39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
39 _category count :  21
39 _aug_num :  85
4 _category count :  2
4 _aug_num :  900
4 _category count :  2
4 _aug_num :  900
40 _category co

53 _category count :  4
53 _aug_num :  450
54 _category count :  3
54 _aug_num :  600
54 _category count :  3
54 _aug_num :  600
54 _category count :  3
54 _aug_num :  600
55 _category count :  2
55 _aug_num :  900
55 _category count :  2
55 _aug_num :  900
56 _category count :  3
56 _aug_num :  600
56 _category count :  3
56 _aug_num :  600
56 _category count :  3
56 _aug_num :  600
57 _category count :  4
57 _aug_num :  450
57 _category count :  4
57 _aug_num :  450
57 _category count :  4
57 _aug_num :  450
57 _category count :  4
57 _aug_num :  450
58 _category count :  3
58 _aug_num :  600
58 _category count :  3
58 _aug_num :  600
58 _category count :  3
58 _aug_num :  600
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _category count :  11
59 _aug_num :  163
59 _

73 _category count :  6
73 _aug_num :  300
73 _category count :  6
73 _aug_num :  300
73 _category count :  6
73 _aug_num :  300
73 _category count :  6
73 _aug_num :  300
74 _category count :  2
74 _aug_num :  900
74 _category count :  2
74 _aug_num :  900
75 _category count :  2
75 _aug_num :  900
75 _category count :  2
75 _aug_num :  900
76 _category count :  3
76 _aug_num :  600
76 _category count :  3
76 _aug_num :  600
76 _category count :  3
76 _aug_num :  600
77 _category count :  2
77 _aug_num :  900
77 _category count :  2
77 _aug_num :  900
78 _category count :  2
78 _aug_num :  900
78 _category count :  2
78 _aug_num :  900
79 _category count :  3
79 _aug_num :  600
79 _category count :  3
79 _aug_num :  600
79 _category count :  3
79 _aug_num :  600
8 _category count :  4
8 _aug_num :  450
8 _category count :  4
8 _aug_num :  450
8 _category count :  4
8 _aug_num :  450
8 _category count :  4
8 _aug_num :  450
80 _category count :  2
80 _aug_num :  900
80 _category count 

### 각 카테고리 별로 augmented 이미지 111개 씩 validation 폴더에 저장

In [ ]:
import random
import shutil
for img_category in list(set(img_category_list)):
    # 해당 category 폴더에서 무작위로 선택된 15개의 이미지 경로를 저장 (경로 지정해줘야 함)
    folder_path = "/content/Training/" + str(img_category)
    augmented_images_pathes = glob.glob(folder_path+"/*.jpeg")
    selected_images_pathes = random.sample(all_images_pathes, 111)
    
    # 이동할 폴더 생성(경로 지정해줘야 함)
    move_to_path = "/content/Validation/"+ str(img_category)
    os.makedirs(move_to_path)
    
    for selected_image_path in selected_images_pathes:
        file_name = selected_image_path.split('/')[-1]
        from_image_path = folder_path + "/" + file_name
        shutil.move(from_image_path, move_to_path)